# Tutorial for HPS multiple DDX4 

In [5]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb
from openabc.utils.insert import insert_molecules

# set simulation platform
platform_name = 'CPU'

Parse a single DDX4.

In [6]:
sequence = 'MGDEDWEAEINPHMSSYVPIFEKDRYSGENGDNFNRTPASSSEMDDGPSRRDHFMKSGFASGRNFGNRDAGECNKRDNTSTMGGFGVGKSFGNRGFSNSRFEDGDSSGFWRESSNDCEDNPTRNRGFSKRGGYRDGNNSEASGPYRRGGRGSFRGCRGGFGLGSPNNDLDPDECMQRTGGLFGSRRPVLSGTGNGDTSQSRSGSGSERGGYKGLNEEVITGSGKNSWKSEAEGGES'
ca_pdb = 'init_DDX4_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse molecule with default settings.


Prepare the initial configuration.

In [7]:
# insert molecules into the simulation box randomly
n_mol = 10
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[50, 50, 50])


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [8]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
protein.create_system(top)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 300*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
platform_name = 'CPU'
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_multi_DDX4.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,2544.915852478538,5918.722447597788,8463.638300076327,201.17526850836057,0
200,2.0000000000000013,2989.8210888020003,5584.903549771207,8574.724638573207,189.82888303446765,63.2
300,2.99999999999998,2973.90902640384,5670.936133890659,8644.845160294499,192.7530999349786,60
400,3.9999999999999587,2895.28295974863,5824.835176782076,8720.118136530706,197.98407360387,59.3
500,4.999999999999938,2752.706858110496,6055.94355098579,8808.650409096286,205.8393649520617,57.9
